## Setup

In [1]:
#test

import arcpy
from datetime import datetime
import os
import sys
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
import sqlalchemy as sa
import pandas as pd
from arcgis import GIS
from arcgis.features import FeatureSet, GeoAccessor, GeoSeriesAccessor, FeatureLayer
import pandas as pd
import numpy as np
import requests

gis = GIS()
# # Set Pandas display options to show all rows and columns
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# in memory output file path
wk_memory = "memory" + "\\"

# set workspace and sde connections 
working_folder = "F:\Research and Analysis\Fisheries\Streams\Bioassessment\California Stream Condition Index\California Stream Condition Index"
workspace      = "C:\GIS\Scratch.gdb"
arcpy.env.workspace = "C:\GIS\Scratch.gdb"

# network path to connection files
filePath = "C:\\GIS\\DB_CONNECT"

# database file path 
sdeBase    = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")

# local variables
fdata = os.path.join(sdeBase, "sde.SDE.Monitoring")
## Final feature class to append to in Enterprise Geodatabase
sdeStreams = os.path.join(sdeBase, "sde.SDE.Monitoring\sde.SDE.Stream")
## orginin CSVs that come from preprocessing or R tools created by State
originalcsv = os.path.join(working_folder,"2022_CSCI","19-20NV-22allcore.csv")
locationcsv = os.path.join(working_folder, "2022_CSCI","Stations19_22.csv")

Field_Mapping={
    'StationCode': 'SITE_NAME',
    'Year': 'YEAR_OF_COUNT',
    'New_Lat': 'LATITUDE',
    'New_Long': 'LONGITUDE',
    'CSCI': 'COUNT_VALUE',
    'STATION_TYPE': 'STATION_TYPE'
    
}


In [6]:
# Function to map values based on the middle three characters
def map_middle_three_chars(value):
    middle_three = value[3:6]
    if middle_three == 'REF':
        return 'Reference'
    # Add more conditions as needed
    elif middle_three == 'TPA':
        return 'Trend Panel A'
    elif middle_three == 'TPB':
        return 'Trend Panel B'
    else:
        return 'Targeted Monitoring'

#Calculate Rating for CSCI value
#Define a function to categorize values based on ranges
def categorize_value(value):
    if 0 <= value < 0.6:
        return 'poor'
    elif 0.6 <= value < 0.8:
        return 'marginal'
    elif 0.8 <= value < 1.0:
        return 'good'
    else:
        return 'excellent'
    
def get_fs_data(service_url):
    
    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
    # Convert the query result to a list of dictionaries
    feature_list = query_result.features

    # Create a pandas DataFrame from the list of dictionaries
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])

    return all_data

In [7]:
df = get_fs_data('https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/8')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   OBJECTID          351 non-null    int64  
 1   STATION_CODE      351 non-null    object 
 2   SITE_NAME         351 non-null    object 
 3   YEAR_OF_COUNT     351 non-null    int64  
 4   LONGITUDE         351 non-null    float64
 5   LATITUDE          351 non-null    float64
 6   COUNT_VALUE       351 non-null    float64
 7   STATION_TYPE      351 non-null    object 
 8   DURATION          351 non-null    object 
 9   LTINFO            351 non-null    object 
 10  GlobalID          351 non-null    object 
 11  created_user      68 non-null     object 
 12  created_date      68 non-null     float64
 13  last_edited_user  68 non-null     object 
 14  last_edited_date  68 non-null     float64
dtypes: float64(5), int64(2), object(8)
memory usage: 41.3+ KB


## Transform CSCI Scores to Point feature class in Enterprise Geodatabase

In [5]:
# create data fraems from csv
dfCSCI=pd.read_csv(originalcsv)
dflocations=pd.read_csv(locationcsv)
# join data frames to get Lat Long in final output
df=dfCSCI.merge(dflocations, how='inner', on='StationCode')

In [6]:
# get year from sample id
df['Year']=df.SampleID.str.split("_").str[-1]

#Calculate Station Type from middle three charcters in station code
df['STATION_TYPE'] = df['StationCode'].apply(map_middle_three_chars)

# Apply the categorization function to create the new field
df['Rating'] = df['CSCI'].apply(categorize_value)

# rename feils based on field mappings
df_final = df.rename(columns=Field_Mapping).drop(columns=[col for col in df.columns if col not in Field_Mapping])

# establish duration field
df_final['DURATION']='Long-term'

# link to the dashboard
df_final['LTINFO']='https://monitoring.laketahoeinfo.org/MonitoringSite/Detail/285'

# station code is site name, site name is station code.
df_final['STATION_CODE']=df_final['SITE_NAME']

# export to csv
df_final.to_csv(os.path.join(working_folder,"StreamCSCI_proccesed.csv"), index= False)

In [7]:
# convert CSV to point feature class
arcpy.management.XYTableToPoint(os.path.join(working_folder,"StreamCSCI_proccesed.csv"), 
                                "NewCSCI_points", 
                                "LONGITUDE", "LATITUDE")

# project to UTM Zone 10N
arcpy.Project_management("NewCSCI_points", "NewStream_CSCI_Projected", 26910)


<Result 'C:\\GIS\\Scratch.gdb\\NewStream_CSCI_Projected'>

In [12]:

inputfc= "NewStream_CSCI_Projected"

# disconnect all users
print("\nDisconnecting all users...")
arcpy.DisconnectUser(sdeBase, "ALL")
 
# unregister the sde feature class as versioned
print ("\nUnregistering feature dataset as versioned...")
arcpy.UnregisterAsVersioned_management(fdata,"NO_KEEP_EDIT","COMPRESS_DEFAULT")
print ("\nFinished unregistering feature dataset as versioned.")

arcpy.management.Append(inputfc, sdeStreams,"NO_TEST")

# disconnect all users
print("\nDisconnecting all users...")
arcpy.DisconnectUser(sdeBase, "ALL")

# register SDE feature class as versioned
arcpy.RegisterAsVersioned_management(fdata, "NO_EDITS_TO_BASE")


Disconnecting all users...

Unregistering feature dataset as versioned...

Finished unregistering feature dataset as versioned.

Disconnecting all users...


<Result 'C:\\GIS\\DB_CONNECT\\Vector.sde\\sde.SDE.Monitoring'>

#### Dev

In [7]:
# Import raw tabular data from csv
csci_data = pd.read_csv('Raw_Data\csci_data.csv')

In [11]:
# Pivot to new format
grouping_columns = ['StationCode','STREAM_NAME', 'COUNTY']
csci_data_flat = csci_data.pivot(index=grouping_columns, columns='SAMPLEDATE',values='CSCI')
csci_data_flat = csci_data_flat.reset_index()

In [ ]:
#Code to create stream csci sampling stations
stream_csci_input_layer = "import"
unique_stream_samples = ""

stations=[]
# Create a search cursor to iterate through the original feature layer
with arcpy.da.SearchCursor(stream_csci_input_layer, ['SITE_NAME']) as cursor:
    for row in cursor:
        value = row[0]
        if value not in stations:
            stations.append(value)

# Create an insert cursor for the output feature layer
with arcpy.da.InsertCursor(output_feature_layer, [unique_column]) as cursor:
    for value in unique_values:
        cursor.insertRow((value,))

# Clean up
del cursor

# Optionally, save the output feature layer to a file
arcpy.CopyFeatures_management(output_feature_layer, "C:/Path/To/Your/Output/FeatureClassFile")

In [ ]:
#New feature class with the sampling station

merged_df = pd.merge(sdfCensus, tdc_flat, on='TRPAID', how='inner')
columns_drop=['GlobalID', 'YEAR', 'created_date', 'created_user', 'last_edited_date', 'last_edited_user', 'Shape.STArea()', 'Shape.STLength()']
merged_df = merged_df.drop(columns=columns_drop)